In [1]:
%config IPCompleter.use_jedi = False

In [4]:
import pandas as pd
train = pd.read_csv('../resources/data/train_ctrUa4K.csv')
test = pd.read_csv('../resources/data/test_lAUu6dG.csv')
train = train.drop('Loan_ID', axis=1)
train.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [5]:
train.shape

(614, 12)

In [6]:
X = train.drop('Loan_Status', axis=1)
y = train['Loan_Status']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

numeric_transformer = Pipeline(steps=[
    ('imputer', imputer),
    ('scaler', scaler)])

cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
cat_onehot = OneHotEncoder(handle_unknown='ignore')

categorical_transformer = Pipeline(steps=[
    ('imputer', cat_imputer),
    ('onehot', cat_onehot)])

In [8]:
numeric_features = train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train.select_dtypes(include=['object']).drop(['Loan_Status'], axis=1).columns
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [9]:
import time

start = time.time()
Xt = preprocessor.fit(X_train)
end = time.time()
print('Time taken: ' + str(end - start))

Time taken: 0.016543149948120117


In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

In [11]:
from sklearn.base import ClassifierMixin
from sklearn.base import BaseEstimator

class ScaleTestEstimator(ClassifierMixin, BaseEstimator):
    num_iters = 100
    classifier : ClassifierMixin = None

    def __init__(self, num_iters, classifier: ClassifierMixin):
        self.num_iters = num_iters
        self.classifier = classifier

    def fit(self, X, y):
        for i in range(self.num_iters):
            self.classifier.fit(X, y)
        return self
            
    def predict(self, X):
        return self.classifier.predict(X)

    def score(self, X, y, sample_weight=None):
        return self.classifier.score(X, y, sample_weight)

In [12]:
Xt = preprocessor.fit_transform(X_train)

In [13]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]

In [14]:
classifiers[0]

KNeighborsClassifier(n_neighbors=3)

In [15]:
c_a = ScaleTestEstimator(50, DecisionTreeClassifier())
c_b = ScaleTestEstimator(50, RandomForestClassifier())
c_c = ScaleTestEstimator(50, GradientBoostingClassifier())
classifiers = [c_a, c_b, c_c]

In [16]:
import sklearn.base as base

In [17]:
base.is_classifier(c_a)

True

In [18]:
base.clone(c_a)

ScaleTestEstimator(classifier=DecisionTreeClassifier(), num_iters=50)

In [19]:
import time
start = time.time()

c_a = ScaleTestEstimator(50, DecisionTreeClassifier())
c_b = ScaleTestEstimator(50, RandomForestClassifier())
c_c = ScaleTestEstimator(50, GradientBoostingClassifier())
classifiers = [c_a, c_b, c_c]

classifier_results=[]
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)
    pipe.predict(X_train)
    
end = time.time()
tt = end - start
print('time taken: ' + str(tt))

time taken: 10.476289987564087


In [20]:
import ray
ray.init()

2021-05-13 09:08:02,853	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '9.160.6.207',
 'raylet_ip_address': '9.160.6.207',
 'redis_address': '9.160.6.207:6379',
 'object_store_address': '/tmp/ray/session_2021-05-13_09-08-02_312724_17276/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-13_09-08-02_312724_17276/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-13_09-08-02_312724_17276',
 'metrics_export_port': 65326,
 'node_id': 'badab7a8f4dec1bbf68f3f5850c314ec7cd20928'}

In [22]:
from hercules.Datamodel import Xy
from hercules.Datamodel import XYRef
import hercules.Datamodel as dm

In [30]:
X_ref = ray.put(X_train)
y_ref = ray.put(y_train)

Xy_ref = XYRef(X_ref, y_ref)
Xy_ref_list = [Xy_ref]

In [24]:
pipeline = dm.Pipeline()

In [25]:
node_a = dm.OrNode('preprocess', preprocessor)
node_b = dm.OrNode('c_a', c_a)
node_c = dm.OrNode('c_b', c_b)
node_d = dm.OrNode('c_c', c_c)

In [26]:
pipeline.add_edge(node_a, node_b)
pipeline.add_edge(node_a, node_c)
pipeline.add_edge(node_a, node_d)

In [27]:
import hercules.RuntimeNew as rt

In [28]:
from hercules.RuntimeNew import ExecutionType

In [31]:
start = time.time()


in_args={node_a: Xy_ref_list}

out_args = rt.execute_pipeline(pipeline, ExecutionType.FIT, in_args)

node_b_out_args = ray.get(out_args[node_b])
node_c_out_args = ray.get(out_args[node_c])
node_d_out_args = ray.get(out_args[node_d])
end = time.time()
print ('Time taken: ' + str(end - start))

Time taken: 9.040623903274536


In [33]:
b_out_xyref = node_b_out_args[0]

In [34]:
ray.get(b_out_xyref.get_Xref())

array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y',
       'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'N', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'N',
       'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'N',
       'Y', 'N', 'N', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N',
       'N', 'N', 'N', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y